In [1]:
%%time
import numpy as np
from numpy.linalg import norm
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import pickle

n_tweets_to_read = 15000 # Choose the number of tweets to read

cosine_similarity = lambda a, b: np.inner(a, b) / norm(a) * norm(b) if norm(a) != 0.0 and norm(b) != 0.0 else 0.0

def TermDocumentMatrix(docs, docIDs=None):
    vectorizer = CountVectorizer(lowercase=True, stop_words=None)
    tdm = vectorizer.fit_transform(docs)
    tdm_feature_names = vectorizer.get_feature_names()
    #
    df = pd.DataFrame(tdm.toarray(), columns=tdm_feature_names, dtype="float64")
    if docIDs is not None:
        df.index = docIDs    
    return df

#Initialisation
ps = PorterStemmer()

# Read the data and remove duplicates if exist
tweets = []
with open("data/data.txt", encoding="utf-8") as file:
    for i, line in enumerate(file):
        if i < n_tweets_to_read:
            tweets.append(line)
        else:
            break  
tweets = list(set(tweets)) # delete duplicates
print("{} unique Tweets loaded\n".format(len(tweets)))

# Data Cleanning and prepocesing phase
cleanTweet = []
tweetIDs = []
tweetsProcessed = []
for tweet in tweets:
    try:
        doc = tweet.split("\t") 
        if len(doc)==7:
            cleanTweet.append(doc[5])
            tweetIDs.append(doc[0])     # add ID to a list
            doc = doc[5:6]   # remove everything that's note the tweet content      
            tok_doc = word_tokenize(" ".join(doc))    # tokenize remaining document
            stemmed_doc = [ps.stem(word) for word in tok_doc] 
            tweetsProcessed.append(" ".join(stemmed_doc))   #stemmed words
    except:
        pass

# TF-IDF construction part

# Terms frequency
tdf = TermDocumentMatrix(tweetsProcessed, tweetIDs)

# Document frequency
documentFrequencies = []
for index, series in tdf.iteritems(): 
    documentFrequencies.append(len(series.nonzero()[0])) 

# TF-IDF weight
tdf.applymap(lambda x: 1.0 + np.log10(x) if x > 0.0 else 0.0) # log frequency weight
idf = pd.Series(np.log10(len(tweets)/np.array(documentFrequencies))) # Inverse document frequency
tf_idf = tdf * idf.values

print(tf_idf)

with open('save.pickle', 'wb') as handle:
    pickle.dump(tf_idf, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('cleantweet.pickle', 'wb') as handle:
    pickle.dump(cleanTweet, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('tweetids.pickle', 'wb') as handle:
    pickle.dump(tweetIDs, handle, protocol=pickle.HIGHEST_PROTOCOL)

def printTopSimilarTweets(tf_idf=tf_idf,tweet='Hello', n=20):

    cleanT = tweet.split()   # tokenize remaining document
    vec = [ps.stem(word) for word in cleanT] 
    tf_idf=tf_idf.append(pd.Series(name='TestTweet'))
    tf_idf=tf_idf.fillna(0)
    for i in vec:
        try:
            tf_idf.loc['TestTweet', i] += 1
        except:
            pass
    a= tf_idf.loc['TestTweet']
    result = tf_idf.apply(lambda row: cosine_similarity(a, row), 
                          axis='columns').sort_values(ascending=False) 
    
    for i in range(0, n):
        try:
            print("{}: ".format(i+1) + cleanTweet[tweetIDs.index(result.index[i])] + "\n")
        except:
            pass

# output
printTopSimilarTweets(tweet="Hopefully the violence & unrest in Charlotte will come to an immediate end")

[nltk_data] Downloading package punkt to /home/paul/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


14951 unique Tweets loaded



/home/paul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:64: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead


        00  000  000th  0062343971  00am  00d3ec5a  00pm  00pme  00ykww_bq2i  \
10494  0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
3658   0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
12406  0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
3002   0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
7879   0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
10993  0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
1263   0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
8211   0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
13104  0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
5743   0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
10626  0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
1676   0.0  0.0    0.0         0.0   0.0

In [ ]:
%%time
printTopSimilarTweets(tweet="We will have to see what Russia's next move will be")

In [2]:
with open('save.pickle', 'rb') as f:
    matrix = pickle.load(f)
    
print(matrix)

        00  000  000th  0062343971  00am  00d3ec5a  00pm  00pme  00ykww_bq2i  \
8635   0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
12417  0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
5990   0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
2983   0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
1589   0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
6539   0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
7433   0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
9855   0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
4407   0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
10699  0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
4491   0.0  0.0    0.0         0.0   0.0       0.0   0.0    0.0          0.0   
7600   0.0  0.0    0.0         0.0   0.0